<a href="https://colab.research.google.com/github/Alx-a-cod/MLiterallyConfused/blob/main/ML_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
 # ==== file upload from PC ====
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes.csv


In [22]:
# ==== libraries import ====
import numpy as np
import pandas as pd
# ==== dependencies import ====
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, roc_auc_score, roc_curve, classification_report)
import matplotlib.pyplot as plt
import seaborn as sns



In [26]:
# ==== load dataset ====

filename = "diabetes.csv"

df = pd.read_csv("diabetes.csv")

# ==== check both English (,) and European (;) separators ====
try:
    df = pd.read_csv(filename, sep=',')
    if df.shape[1] == 1: # if everything gets lumped in one column
        raise ValueError("Possible separator issue. Check your file and retry?")
except Exception:
    df = pd.read_csv(filename, sep=';')

# === dataset overview ===

print(f"Dataset loaded successfully: {filename}")
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")

print("Columns:")
print(df.columns.tolist(), "\n")

print("Missing values per column:")
print(df.isna().sum(), "\n") # returns a DataFrame of booleans: True if it’s NaN, False if it’s a valid number or string

print("Preview:")
pd.set_option('display.max_columns', None)
df.head()

Dataset loaded successfully: diabetes.csv
Shape: 1199 rows × 9 columns

Columns:
['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'] 

Missing values per column:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64 

Preview:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.63,50,1
1,1,85,66,29,0,26.6,0.35,31,0
2,8,183,64,0,0,23.3,0.67,32,1
3,1,89,66,23,94,28.1,0.17,21,0
4,0,137,40,35,168,43.1,2288.00,33,1


In [13]:
# ==== statistics & health-check about the Dataset ====

summary = df.describe().T
summary['missing'] = df.isna().sum()
display(summary.round(2)) # display wrap for prettier dataframes: renders as clean HTML table in Colab instead of print text dump


,count,mean,std,min,25%,50%,75%,max,missing
Pregnancies,1199.0,4.62,3.11,0.00,2.0,4.00,7.00,17.0,0
Glucose,1199.0,137.64,33.58,44.00,109.0,143.00,164.00,199.0,0
BloodPressure,1199.0,76.18,12.07,24.00,68.0,76.00,84.50,122.0,0
SkinThickness,1199.0,30.80,7.74,7.00,27.0,31.00,35.00,99.0,0
Insulin,1199.0,143.03,69.93,14.00,120.0,135.00,140.00,846.0,0
BMI,1199.0,33.43,6.22,18.20,29.8,33.20,37.45,67.1,0
DiabetesPedigreeFunction,1199.0,52.54,265.24,0.08,0.3,0.62,0.72,2329.0,0
Age,1199.0,40.10,14.01,21.00,26.5,41.00,51.00,81.0,0
Outcome,1199.0,0.40,0.49,0.00,0.0,0.00,1.00,1.0,0


In [34]:
import numpy as np
import pandas as pd

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# ==== 0. original copy to avoid overwriting ====

df_clean = df.copy()

# ==== 1. Replaced impossible zeros with NaN ====
# Define variables used in the original cell
target = "Outcome"
zeros_as_missing = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "Age"]
log_transform = [] # Assuming no log transform is needed for now.

if zeros_as_missing is not None:
    for col in zeros_as_missing:
        if col in df_clean.columns:
            missing_count = (df_clean[col] == 0).sum()
            df_clean[col] = df_clean[col].replace(0, np.nan)
            print(f"Replaced {missing_count} zeros in '{col}' with NaN.")

    print("\nMissing values after zero replacement:")
    display(df_clean[zeros_as_missing].isna().sum().to_frame("Missing Values"))

    # Optional log-transform for skewed features
    if log_transform:
        for col in log_transform:
            if col in df_clean.columns:
                df_clean[col] = np.log1p(df_clean[col])
                print(f"Applied log1p transform to '{col}'")

# === 1.5 : Splitting dataset before imputation/scaling (leakage prevention)===

X = df_clean.drop(target, axis=1)
y = df_clean[target]

# 1st split: train+val vs test
x_trainval, x_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.15, stratify=y, random_state=42
)

# 2nd split: train vs val (approx. 70/15/15)
x_train, x_val, y_train, y_val = train_test_split(
    x_trainval, y_trainval, test_size=0.1765, stratify=y_trainval, random_state=42
)

print(f"\nData split shapes: Train: {x_train.shape}, Val: {x_val.shape}, Test: {x_test.shape}")

# === 2: build preprocessing pipeline (imputation + scaling) ===

numeric_features = X_train.columns.tolist()

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")), # median imputation
    ("scaler", StandardScaler())                   # standard scaling

    # Uncomment PCA if you want dimensionality reduction
    # ("pca", PCA(n_components=0.95))  # keeps 95% variance
])

preprocessor = ColumnTransformer([
("num", numeric_transformer, numeric_features)],
remainder="drop")

# Fit/transform on training data only
x_train_p = preprocessor.fit_transform(x_train)
x_val_p = preprocessor.transform(x_val)
x_test_p = preprocessor.transform(x_test)

print("\nPreprocessing pipeline applied (median imputation + standard scaling).")

# === 3: outlier detection ===
# z-score

X_train_df = pd.DataFrame(x_train_p, columns=numeric_features)
z_scores = np.abs(stats.zscore(X_train_df))
outlier_flags_train = (z_scores > 3).any(axis=1)

print(f"Training rows flagged as potential outliers (|z|>3): {outlier_flags_train.sum()}")


# Last sanity check
print("\n Quick summary after cleaning:")
display(pd.DataFrame(x_train_p, columns=numeric_features).describe().T.round(2))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

Added:

*   Count to se how many zeros are replaced - because it's always good to see   
    what's being fixed. Domain-specific missing flags.
    - Why? ---> ML algorithms treat zero as real data, which biases results.Replacing zeros with NaN correctly flags them as missing, so they can be handled later by imputation.
    - Result: a dataset that explicitly marks missing data instead of hiding it as zeros.
    ____
*   Split train/val/test. Must happen before any fit.
    - Training set (~70%) → Used to fit the preprocessing pipeline and train the model.
    - Validation set (~15%) → Used to tune hyperparameters.
    - Test set (~15%) → Used to evaluate final performance.

    You split before doing imputation or scaling to prevent data leakage.
    If  imputed or scaled before splitting, info from the test set would influence training — artificially inflating model performance.  

    - Result: Clean, isolated data splits for unbiased model evaluation.
    ____

*   Imputed missing values. Done in pipeline to prevent leakage.
    Built a data transformation pipeline that:
    - Imputes missing values using the median of each column.
    - Median is robust to outliers (unlike mean).
    - Standardizes all features with StandardScaler.
    - Each feature gets mean = 0 and standard deviation = 1.
    - combined this pipeline in a ColumnTransformer that applies it to all numeric features.
    
    Why:
    - The Pipeline ensures transformations happen in the right order and only on training data.
    - The ColumnTransformer makes it scalable if later we get categorical features.
    - Using fit_transform only on x_train and transform on x_val/x_test avoids data leakage again. Kinda.
    
    Result:
    - All numeric columns are imputed and standardized, consistently across train/val/test.

*   Outliers detection ---> to help identify suspicious   
    data. (i.e. that insulin over 800) Only if we actually filter or cap.
    - Can compute the z-score of each numeric value:
      𝑧 = (𝑥−𝜇) / 𝜎
      
      and count how many observations per column exceed |z| > 3 (rough heuristic for outliers).
    
    Why:
    - we are not removing outliers here — just detecting them for analysis
      and applying it on the training set (important!), so test set doesn’t influence detection thresholds.
      
    Result:
    - We can see how many potential outliers each column has, which can help   
      later decide whether to clip or leave them.

*   Final Re-summarization at the end so to confirm data is clean. Diagnostic.



In [ ]:
# wrapped into a function, because yes

def prepare_data(df, target_col="Outcome", test_size=0.15, random_state=42, use_pca=False):
    zero_as_missing = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", ]
    for col in zero_as_missing:
        df[col] = df[col].replace(0, np.nan)

    X = df.drop(target_col, axis=1)
    y = df[target_col]

    # Splits
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval, test_size=0.1765, stratify=y_trainval, random_state=random_state
    )

    # Preprocessing pipeline
    numeric_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    if use_pca:
        numeric_transformer.steps.append(("pca", PCA(n_components=0.95)))

    preprocessor = ColumnTransformer([
        ("num", numeric_transformer, X.columns)
    ])

    # Fit + transform
    X_train_p = preprocessor.fit_transform(X_train)
    X_val_p = preprocessor.transform(X_val)
    X_test_p = preprocessor.transform(X_test)

    return X_train_p, X_val_p, X_test_p, y_train, y_val, y_test, preprocessor